# Predict Turbofan Degradation

In this example, we build a machine learning application that predicts turbofan engine degradation. This application is structured into three important steps:

* Prediction Engineering
* Feature Engineering
* Machine Learning

In the first step, we generate our own labels from the data by using [Compose](https://compose.alteryx.com/). In the second step, we generate features for the labels by using [Featuretools](https://docs.featuretools.com/). In the third step, we search for the best machine learning pipeline by using [EvalML](https://evalml.alteryx.com/). 
After working through these steps, you will learn how to build machine learning applications for real-world problems like predictive maintenance. Let's get started.

In [ ]:
from demo.predict_rul import load_sample
from matplotlib.pyplot import subplots
import composeml as cp
import featuretools as ft
import evalml

We will use a dataset provided by NASA simulating turbofan engine degradation. In this dataset, we have engines which are monitored over time. Each engine had operational settings and sensor measurements recorded for each cycle. The remaining useful life (RUL) is the amount of cycles an engine has left before it needs maintenance. What makes this dataset special is that the engines run all the way until failure, giving us precise RUL information for every engine at every point in time. 

In [ ]:
df = load_sample()

df.head()

## Prediction Engineering

> Which range is the RUL of a turbofan engine in?

In this prediction problem, we want to group the RUL into ranges. Then, predict which range the RUL is in. We can make variations of the ranges to create different prediction problems. For example, the ranges can be manually defined (0 - 150, 150 - 300, etc.) or based on the quartiles from historical observations. These variations can be done by simply binning the RUL. This helps us explore different scenarios which is crucial for making better decisions.

### Defining the Labeling Process

Let's stary by defining the labeling function of an engine that calculates the RUL. Given that engines run all the way until failure, the RUL is just the remaining number of observations.

In [ ]:
def rul(ds):
    return len(ds) - 1

### Representing the Prediction Problem

Then, let's represent the prediction problem by creating a label maker with the following parameters:

* The `target_entity` as the engine ID, because we want to generate labels for each engine.
* The `labeling_function` as the function we defined previously.
* The `time_index` as the event time.

In [ ]:
lm = cp.LabelMaker(
    target_entity='engine_no',
    labeling_function=rul,
    time_index='time',
)

### Finding the Training Examples

Now, let's run a search to get the RUL using the following parameters:

* The historical data sorted by the event time.
* The `num_examples_per_instance` as the number of training examples to find per engine. In this search, we find up to 20 examples per enigne.
* The `minimum_data` as the first number of cycles to skip before the labeling starts. In this data sample, turbines generally don't fail before 5 cycles, so we start after 5 cycles.
* The `gap` as the number of cycles to place between examples. This is done to cover different points in time of an engine.

We can easily tweak the search parameters and generate more training examples as the requirements of our model changes.

In [ ]:
lt = lm.search(
    df.sort_values('time'),
    num_examples_per_instance=20,
    minimum_data=5,
    gap=20,
    verbose=False,
)

lt.head()

The output from the search is a label times table with three columns:

* The engine ID associated to the RUL.
* The point in time of the engine when we calculated the RUL. This is also known as a cutoff time for building features. Only data that existed beforehand is valid to use for making predictions about the outcome.
* The value of the RUL. This is calculated by our labeling function.

At this point, we only have continuous values of the RUL. As a helpul reference, we can print out the search settings that were used to generate the labels.

In [ ]:
lt.describe()

We can also get a better look at the values by plotting the distribution and the cumulative count across time.

In [ ]:
%matplotlib inline
fig, ax = subplots(nrows=2, ncols=1, figsize=(6, 8))
lt.plot.distribution(ax=ax[0])
lt.plot.count_by_time(ax=ax[1])
fig.tight_layout(pad=2)

Next, let's bin the values into ranges for discrete labels. We can instantly create variations of the prediction problem by exploring different ranges without running the search again for the training examples. In this case, we will just use ranges based on the quartiles.

In [ ]:
lt = lt.bin(4, quantiles=True, precision=0)

When we print out the settings again, we can see that the description of the labels has been updated and reflects the recent changes.

In [ ]:
lt.describe()

Let's see the new label distribution and the cumulative count across time.

In [ ]:
%matplotlib inline
fig, ax = subplots(nrows=2, ncols=1, figsize=(6, 8))
lt.plot.distribution(ax=ax[0])
lt.plot.count_by_time(ax=ax[1])
fig.tight_layout(pad=2)

## Generate Features

Now, we are ready to generate features for our prediction problem.

### Create Entity Set

To get started, let's create an entity set for the observations.

In [ ]:
es = ft.EntitySet('observations')

es.entity_from_dataframe(
    dataframe=df.reset_index(),
    entity_id='recordings',
    index='id',
    time_index='time',
)

es.normalize_entity(
    base_entity_id='recordings',
    new_entity_id='engines',
    index='engine_no',
)

es.normalize_entity(
    base_entity_id='recordings',
    new_entity_id='cycles',
    index='time_in_cycles',
)

es.plot()

### Create Feature Matrix

Let's generate the features that correspond to our labels. To do this, we set the engines as the target entity and our labels as the cutoff time. This way the features are calculated only using data up to the cutoff time of each label. Notice that the output from Compose integrates easily with Featuretools.

In [ ]:
fm, fd = ft.dfs(
    entityset=es,
    target_entity='engines',
    agg_primitives=['sum'],
    trans_primitives=[],
    cutoff_time=lt,
    cutoff_time_in_index=True,
    include_cutoff_time=False,
    verbose=False,
)

fm.head()

## Machine Learning

Now, we are ready to create a machine learning model for our prediction problem.

### Split Data

Let's extract the labels from the feature matrix and split the data into training and holdout sets.

In [ ]:
y = fm.pop('rul').cat.codes
splits = evalml.preprocessing.split_data(fm, y, test_size=0.2, random_state=0)
X_train, X_holdout, y_train, y_holdout = splits

### Train Model

Next, we search for the optimal pipeline by trying out different models on the training set.

In [ ]:
automl = evalml.AutoMLSearch(problem_type='multiclass', objective='f1_macro')
automl.search(X_train, y_train, data_checks=None, show_iteration_plot=False)

In [ ]:
automl.best_pipeline.describe()
automl.best_pipeline.graph()

### Test Model

Finally, we score the model performance by evaluating predictions on the holdout set.

In [ ]:
best_pipeline = automl.best_pipeline.fit(X_train, y_train)
score = best_pipeline.score(X_holdout, y_holdout, objectives=['f1_macro'])
dict(score)